In [13]:
from pid_edited_pipeline import run
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scenarios import *
from NL_to_STL import *
from spec_check import *
from visualization import *

In [14]:
# Parameters
max_acc = 10                            # maximum acceleration in m/s^2
max_speed = 0.5                         # maximum speed in m/s
T = 20                                  # time horizon in seconds 
dt = 0.5                                # time step in seconds
N = int(T/dt)                           # total number of time steps
scenario = "treasure_hunt"              # scenario: "reach_avoid", "narrow_maze", or "treasure_hunt"

In [15]:
# Set up scenario
scenarios = Scenarios()
objects = scenarios.get_objects(scenario)
x0 = scenarios.get_starting_state(scenario)
print("x0: ", x0)

x0:  [ 3.  -4.   0.5  0.   0.   0. ]


In [16]:
translator = NL_to_STL(objects, N, dt, print_instructions=True)
spec = 'STL_formulas.inside_cuboid(objects["door_key"]).eventually(0, 40) & STL_formulas.outside_cuboid(objects["west_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["above_door_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["north_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["south_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["NE_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["chest"]).always(0, 40) & STL_formulas.inside_cuboid(objects["region_bounds"]).always(0, 40)'
print("spec: ", spec)

spec:  STL_formulas.inside_cuboid(objects["door_key"]).eventually(0, 40) & STL_formulas.outside_cuboid(objects["west_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["above_door_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["north_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["south_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["NE_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["chest"]).always(0, 40) & STL_formulas.inside_cuboid(objects["region_bounds"]).always(0, 40)


In [17]:
solver = STLSolver(spec, objects, x0, T)
x,u = solver.generate_trajectory(dt, max_acc, max_speed, verbose=False, include_dynamics=True)
spec_checker = Spec_checker(objects, x, N, dt)
inside_objects_array = spec_checker.get_inside_objects_array()
# visualizer = Visualizer(x, objects, animate=False)
# fig, ax = visualizer.visualize_trajectory()
# plt.pause(1)
# fig, ax = spec_checker.visualize_spec(inside_objects_array)
# plt.pause(1)

spec used in the solver:  STL_formulas.inside_cuboid(objects["door_key"]).eventually(0, 40) & STL_formulas.outside_cuboid(objects["west_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["above_door_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["north_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["south_mid_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["NE_inside_wall"]).always(0, 40) & STL_formulas.outside_cuboid(objects["chest"]).always(0, 40) & STL_formulas.inside_cuboid(objects["region_bounds"]).always(0, 40)
<class 'gurobipy.MQuadExpr'>


In [18]:
waypoints = x[:3].T
print("waypoints shape: ", waypoints.shape)

waypoints shape:  (41, 3)


In [19]:
N_waypoints = waypoints.shape[0]
N_extra_points = 5 # extra waypoints to add between waypoints linearly

# Add extra waypoints
total_points = N_waypoints + (N_waypoints-1)*N_extra_points
TARGET_POS = np.zeros((total_points,3))
TARGET_POS[0] = waypoints[0]
for i in range(N_waypoints-1):
    # print("i: ", i, ". (1+N_extra_points)*i: ", (1+N_extra_points)*i)
    TARGET_POS[(1+N_extra_points)*i] = waypoints[i]
    # print("TARGET_POS: ", TARGET_POS[(1+N_extra_points)*i])
    for j in range(N_extra_points+1):
        # print("j: ", j, ". (1+N_extra_points)*i + j: ", (1+N_extra_points)*i + j + 1)
        k = (j+1)/(N_extra_points+1)
        TARGET_POS[(1+N_extra_points)*i + j] = (1-k)*waypoints[i] + k*waypoints[i+1]
        # print("TARGET_POS: ", TARGET_POS[(1+N_extra_points)*i + j])

TARGET_POS.shape


(241, 3)

In [20]:
T = 20
INIT_RPYS = np.array([[0, 0, 0]])

In [23]:
run(waypoints=TARGET_POS, 
    initial_rpys=INIT_RPYS,    
    objects=objects,
    duration_sec=T,
)

[INFO] BaseAviary.__init__() loaded parameters from the drone's .urdf:
[INFO] m 0.027000, L 0.039700,
[INFO] ixx 0.000024, iyy 0.000024, izz 0.000032,
[INFO] kf 0.000000, km 0.000000,
[INFO] t2w 2.250000, max_speed_kmh 30.000000,
[INFO] gnd_eff_coeff 11.368590, prop_radius 0.023135,
[INFO] drag_xy_coeff 0.000001, drag_z_coeff 0.000001,
[INFO] dw_coeff_1 2267.180000, dw_coeff_2 0.160000, dw_coeff_3 -0.110000
viewMatrix (0.9396926760673523, 0.0, 0.3420201241970062, 0.0, 0.3420201241970062, 0.0, -0.9396926760673523, 0.0, -0.0, 1.0000001192092896, -0.0, 0.0, -0.0, -0.0, -13.000000953674316, 1.0)
projectionMatrix (0.7499999403953552, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0000200271606445, -1.0, 0.0, 0.0, -0.02000020071864128, 0.0)
